. Выполнить задания:
1.	Нормализовать базу данных относительно актеров. Выделить таблицу с актерами, а также таблицу многие-ко-многим `актер-фильм`. (Т.е. будут 3 таблицы: актеры, фильмы, актер-фильм. Сохранить новую бд как ‘task1.sqlite’.
2.	Исходя из этой таблицы, средствами SQL, вычислить наболее часто работающую друг с другом пару актеров.


Добавим библиотеки

In [20]:
import sqlite3
import re

Создадим базу данных и назовем ее 'task1.sqlite'
 
Затем создадим курсор, который используется для выполнения операций базы данных, таких как выполнение SQL-запросов и извлечение данных.

In [21]:
conn = sqlite3.connect('task1.sqlite') 
cursor = conn.cursor()

Создадим таблицы actors (для актеров), movies (фильмы), actor_movie (таблица-связь многие ко многим актер-фильм)

In [ ]:

cursor.execute('''CREATE TABLE actors (
                id INTEGER PRIMARY KEY,
                name TEXT )
''')


In [ ]:

cursor.execute('''CREATE TABLE movies (
                id INTEGER PRIMARY KEY,
                title TEXT )
''')

cursor.execute('''CREATE TABLE actor_movie ( 
               actor_id INTEGER, 
               movie_id INTEGER, 
               FOREIGN KEY (actor_id) REFERENCES actors(id), 
               FOREIGN KEY (movie_id) REFERENCES movies(id) )
''')

# После создания таблиц происходит фиксация изменений в базе данных с помощью метода
conn.commit()

# и закрытие соединения с базой данных с помощью
conn.close()

Достаем информацию

In [24]:
# код создает подключение к базе данных SQLite с именем "netflix.sqlite"
connect_netflix = sqlite3.connect('netflix.sqlite')

# затем создает курсор для выполнения операций базы данных
cursor_netflix = connect_netflix.cursor()

# Затем выполняется запрос SQL для извлечения данных из таблицы "netflix_titles"
cursor_netflix.execute('select title, "cast" from netflix_titles')

# Получаем результаты запроса и сохраняем их в переменную
data = cursor_netflix.fetchall()

# После этого соединение с базой данных закрывается.
connect_netflix.close()

Преобразуем в удобный вид

In [25]:
# Создаем новый список с именем "data_new"
data_new = []

# и пустое множество "unique_actors" для зранения имен актеров
unique_actors = set()

# Для каждой строки...
for row in data:
    # извлекается название фильма и строка с актерами.
    movie, cast_str = row
    
    # Строка с актерами разбивается с использованием регулярного выражения на отдельные имена актеров...
    cast = re.split(r' ,|, |,', cast_str)

    # ...которые затем добавляются в множество "unique_actors".
    unique_actors = unique_actors.union(set(cast))

    # данные о фильме и актерах добавляются в список "data_new" в виде кортежей (название фильма, список актеров).
    data_new.append((movie, cast))

Заполнение фильмами

In [26]:
# подключение к базе данных SQLite с именем "task1.sqlite"
connect_movies = sqlite3.connect('task1.sqlite')

# создаем курсор для выполнения операций базы данных.
cursor_movies = connect_movies.cursor()

# Для каждой строки...
for row in data_new:
    # извлекается название фильма и список актеров. 
    movie, actors = row
    
    # Затем выполняется запрос SQL для вставки данных о фильме в таблицу "movies" 
    # с использованием метода "execute" курсора. 
    cursor_movies.execute("INSERT INTO movies VALUES (NULL, ?)", (movie,))

# После завершения итерации происходит фиксация изменений в базе данных с помощью метода "commit()"
connect_movies.commit()

# и закрытие соединения с базой данных с помощью "close()".
connect_movies.close()

Заполение актерами

In [27]:
# подключение к базе данных SQLite с именем "task1.sqlite"
connect_actors = sqlite3.connect('task1.sqlite')

# создаем курсор для выполнения операций базы данных. 
cursor_actors = connect_actors.cursor()

# Затем происходит итерация по уникальным актерам из множества "unique_actors"
for actor in unique_actors:
    # Для каждого актера выполняется запрос SQL для вставки его имени в таблицу "actors"
    # с использованием метода "execute" курсора.
    cursor_actors.execute("INSERT INTO actors VALUES (NULL, ?)", (actor,))

# После завершения итерации происходит фиксация изменений в базе данных с помощью метода "commit()"
connect_actors.commit()

# и закрытие соединения с базой данных с помощью "close()"
connect_actors.close()

Соединение многие-ко-многим

In [28]:
# подключение к базе данных SQLite с именем "task1.sqlite"
connect_task1 = sqlite3.connect('task1.sqlite')

# создаем три курсора для выполнения операций базы данных:
cursor_movies = connect_task1.cursor()
cursor_actors = connect_task1.cursor()
cursor_actor_movie = connect_task1.cursor()

# Затем происходит итерация по данным из переменной "data_new".
for i in data_new:
    # Для каждого фильма и его актеров выполняется запрос SQL, 
    # чтобы получить идентификатор фильма и идентификаторы соответствующих актеров из таблиц "movies" и "actors".
    movie, actors = i
    cursor_movies.execute('SELECT id FROM movies WHERE title = ?', (movie,))
    movie_id = cursor_movies.fetchone()
    for actor in actors:
        cursor_actors.execute('SELECT id FROM actors WHERE name = ?', (actor,))
        actor_id = cursor_actors.fetchone()

        # Затем выполняется запрос SQL для вставки соответствия между актером и фильмом 
        # в таблицу "actor_movie" с использованием метода "execute" курсора.  
        cursor_actor_movie.execute("INSERT INTO actor_movie (actor_id, movie_id) VALUES (?, ?)", (actor_id[0], movie_id[0])) 
        # добавление записи в таблицу

# После завершения итерации происходит фиксация изменений в базе данных с помощью метода "commit()"
connect_task1.commit()

# закрытие соединения с базой данных с помощью "close()".
connect_task1.close()

Поиск актеров 

Этот код выполняет запрос к базе данных SQLite, чтобы найти пару актеров, которые чаще всего снимались вместе.

In [29]:
# Создается подключение к базе данных SQLite с именем "task1.sqlite"
conn = sqlite3.connect('task1.sqlite')
# создается курсор для выполнения операций базы данных.
cursor = conn.cursor()

# С помощью метода "execute" курсора выполняется запрос SQL, который находит пару актеров,
# сыгравших вместе в наибольшем количестве фильмов.
# (Запрос включает операции объединения таблиц, группировки и сортировки)
cursor.execute('''
    SELECT a1.name AS actor1_name, a2.name AS actor2_name, COUNT(*) AS movies_together
    FROM actor_movie am1
    JOIN actor_movie am2 ON am1.movie_id = am2.movie_id AND am1.actor_id < am2.actor_id
    JOIN actors a1 ON am1.actor_id = a1.id
    JOIN actors a2 ON am2.actor_id = a2.id
    GROUP BY actor1_name, actor2_name
    ORDER BY movies_together DESC
    LIMIT 1
''')

# Результат запроса извлекается с помощью метода "fetchone()" и сохраняется в переменной "result".
result = cursor.fetchone()

# Результат выводится на экран, отображая имена актеров и количество фильмов, в которых они снимались вместе.
print("Самая часто работающая друг с другом пара актеров:")
print(f"{result[0]} и {result[1]} - сыграли вместе в {result[2]} фильмах")

# После вывода результатов соединение с базой данных закрывается с помощью метода "close()".
conn.close()

Самая часто работающая друг с другом пара актеров:
John Paul Tremblay и Robb Wells - сыграли вместе в 15 фильмах
